In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
pharma_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_begs.csv')
test_new = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_begs.csv')


In [5]:
pharma_data.drop('ID_Patient_Care_Situation',axis=1,inplace=True)

ind=test_new.index
test_new.drop(['ID_Patient_Care_Situation'],axis=1,inplace=True)

In [6]:
df= pharma_data[pharma_data['Patient_Smoker']=='Cannot say']
print(df.index)
pharma_data.drop(df.index,inplace=True)

pharma_data.Treated_with_drugs=pharma_data.Treated_with_drugs.str.rstrip(' ')

test_new.Treated_with_drugs=test_new.Treated_with_drugs.str.rstrip(' ')

Int64Index([4, 218, 2540, 2711, 4308, 4887, 6308, 10388, 10663, 12782, 18630,
            19123, 21705],
           dtype='int64')


In [7]:
pharma_data.Treated_with_drugs.fillna(pharma_data.Treated_with_drugs.mode()[0],inplace=True)

import statistics as st
col=['A','B','C','D','E','F','Z']
for i in col:
  pharma_data[i].fillna(st.mode(pharma_data[i]),inplace=True)

pharma_data.Number_of_prev_cond=pharma_data.A+pharma_data.B+pharma_data.C+pharma_data.D+pharma_data.E+pharma_data.F+pharma_data.Z

In [8]:
from sklearn.preprocessing import LabelEncoder
object_cols=pharma_data.select_dtypes(exclude=np.number).columns

for col in object_cols:
  le=LabelEncoder()
  pharma_data[col]=le.fit_transform(pharma_data[col])
  test_new[col]=le.transform(test_new[col])

In [9]:
X=pharma_data.drop('Survived_1_year',axis=1)
y=pharma_data['Survived_1_year']

In [10]:

# standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()   
X = scaler.fit_transform(X)
test_new=scaler.transform(test_new)

In [11]:
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.24,random_state=0)

In [22]:
from xgboost import XGBClassifier
model = XGBClassifier( 
                      scale_pos_weight=1,
                      learning_rate=0.2,  
                      colsample_bytree = 0.3,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=1183, 
                      reg_alpha = 0.3,
                      max_depth=8, 
                      gamma=10,
                      colsample_bylevel=0.9,
                     reg_lambda=3,
                      min_child_weight=4
                      
                      )

In [21]:
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

from sklearn .metrics import accuracy_score,f1_score
print(accuracy_score(y_test,y_pred))
f1_score(y_test,y_pred)

0.844612885760693


0.8795298726738492

In [14]:
predictions=model.predict(test_new)

In [15]:
res = pd.DataFrame(predictions)

res.index = ind # its important for comparison
res.columns = ["prediction"]
res.to_csv("prediction_results_HP.csv")  

from google.colab import files
files.download('prediction_results_HP.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>